##Unizipping Dataset


In [1]:
!unzip '/content/drive/MyDrive/archive.zip'

Archive:  /content/drive/MyDrive/archive.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: test_da

##Data Augmentation

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_gen =  ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range = 0.2)
test_gen =  ImageDataGenerator(rescale=1./255) 

In [4]:
train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(224,224),
                                      class_mode = 'categorical',
                                      batch_size=8)

test = train_gen.flow_from_directory('/content/test_data/test_data',
                                      target_size=(224,224),
                                      class_mode = 'categorical',
                                      batch_size=8)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


##CNN

In [5]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

In [9]:
model = Sequential()
model.add(Convolution2D(30,(3,3),20,activation = 'relu',input_shape =(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(80,activation='relu'))
model.add(Dense(16,activation='softmax'))

In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(train,batch_size=8,validation_data = test,epochs=10)

Epoch 1/10
19/19 [==============================] - 93s 5s/step - loss: 2.7565 - accuracy: 0.1067 - val_loss: 2.6967 - val_accuracy: 0.1274
Epoch 2/10
19/19 [==============================] - 120s 7s/step - loss: 2.6804 - accuracy: 0.1600 - val_loss: 2.6724 - val_accuracy: 0.1529
Epoch 3/10
19/19 [==============================] - 120s 7s/step - loss: 2.6303 - accuracy: 0.1733 - val_loss: 2.6628 - val_accuracy: 0.1656
Epoch 4/10
19/19 [==============================] - 90s 5s/step - loss: 2.5634 - accuracy: 0.1933 - val_loss: 2.6317 - val_accuracy: 0.1592
Epoch 5/10
19/19 [==============================] - 91s 5s/step - loss: 2.5077 - accuracy: 0.2067 - val_loss: 2.6253 - val_accuracy: 0.2038
Epoch 6/10
19/19 [==============================] - 120s 7s/step - loss: 2.4187 - accuracy: 0.2667 - val_loss: 2.6140 - val_accuracy: 0.1911
Epoch 7/10
19/19 [==============================] - 89s 5s/step - loss: 2.3087 - accuracy: 0.2667 - val_loss: 2.6334 - val_accuracy: 0.1720
Epoch 8/10
19/19 

## VGG16 Model - For Better Acccuracy

In [11]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [12]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 4s 0us/step


In [13]:
for layer in vgg.layers:
  layer.trainable = False


In [16]:
x = Flatten()(vgg.output)

In [17]:
prediction = Dense(16,activation='softmax')(x)

In [18]:
model2 = Model(inputs=vgg.input,outputs=prediction)

In [19]:
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model2.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))

<ipython-input-20-93be31fd6a36>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))


Epoch 1/10
19/19 [==============================] - 126s 7s/step - loss: 3.8251 - accuracy: 0.2133 - val_loss: 3.6567 - val_accuracy: 0.1847
Epoch 2/10
19/19 [==============================] - 93s 5s/step - loss: 1.0449 - accuracy: 0.7400 - val_loss: 3.1934 - val_accuracy: 0.2675
Epoch 3/10
19/19 [==============================] - 90s 5s/step - loss: 0.4243 - accuracy: 0.8667 - val_loss: 3.1143 - val_accuracy: 0.3312
Epoch 4/10
19/19 [==============================] - 91s 5s/step - loss: 0.2208 - accuracy: 0.9467 - val_loss: 3.0072 - val_accuracy: 0.3758
Epoch 5/10
19/19 [==============================] - 95s 5s/step - loss: 0.1260 - accuracy: 0.9533 - val_loss: 3.0596 - val_accuracy: 0.4076
Epoch 6/10
19/19 [==============================] - 98s 5s/step - loss: 0.0802 - accuracy: 0.9867 - val_loss: 3.0477 - val_accuracy: 0.3631
Epoch 7/10
19/19 [==============================] - 93s 5s/step - loss: 0.0753 - accuracy: 0.9733 - val_loss: 2.9142 - val_accuracy: 0.4395
Epoch 8/10
19/19 [=

##Testing


In [14]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [23]:
output = ['blasti',
 'bonegl',
 'brhkyt',
 'cbrtsh',
 'cmnmyn',
 'gretit',
 'hilpig',
 'himbul',
 'himgri',
 'hsparo',
 'indvul',
 'jglowl',
 'lbicrw',
 'mgprob',
 'rebimg',
 'wcrsrt']

In [28]:
##Prediction using CNN Model
img = image.load_img("/content/DSC_6382.jpg",target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred1 =  np.argmax(model.predict(img))
print(output[pred1])

1/1 [==============================] - 0s 120ms/step
blasti


In [32]:
##Prediction using CNN Model
img1 = image.load_img("/content/IMG_5386.JPG",target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred1_1 =  np.argmax(model.predict(img1))
print(output[pred1_1])

1/1 [==============================] - 0s 22ms/step
himgri


In [29]:
##Prediction using VGG16 Model
img = image.load_img("/content/DSC_6382.jpg",target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred2 =  np.argmax(model2.predict(img))
print(output[pred2])

1/1 [==============================] - 1s 696ms/step
blasti
